# Overview

This Notebook demonstrates a minimal example of querying the cockroach database using the engine, session, and models defined in the backend code.  The backend code is being read-only volume mounted to `/home/jovyan/backend`, and that directory has been added to the `PYTHONPATH` environment variable.  Imports should "just work".

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import logging

logging.basicConfig()
logger = logging.getLogger("sqlalchemy.engine")
logger.propagate = True
logger.setLevel(logging.INFO)

<IPython.core.display.Javascript object>

# Show backend engine

In [3]:
import sqlalchemy as sa
import rich

from app.db import engine, db_session
from app.models import User, Todo

engine

Engine(cockroachdb://root@cockroach:26257/defaultdb?sslmode=disable)

<IPython.core.display.Javascript object>

# Query all Users

In [4]:
with db_session() as session:
    statement = sa.select(User)
    results = session.execute(statement)
    users = results.scalars().all()

users

INFO:sqlalchemy.engine.Engine:select current_schema()
INFO:sqlalchemy.engine.Engine:[raw sql] {}
INFO:sqlalchemy.engine.Engine:select version()
INFO:sqlalchemy.engine.Engine:[generated in 0.00126s] {}
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%(val)s)
INFO:sqlalchemy.engine.Engine:[generated in 0.00113s] {'val': 'sqlalchemy-cockroachdb 1.4.2'}
INFO:sqlalchemy.engine.Engine:SELECT crdb_internal.increment_feature_counter(%(val)s)
INFO:sqlalchemy.engine.Engine:[cached since 0.003788s ago] {'val': 'sqlalchemy 1.4'}
INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT "user".id, "user".created_at, "user".name, "user".password 
FROM "user"
INFO:sqlalchemy.engine.Engine:[generated in 0.00131s] {}
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [5]:
rich.print(users[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7feceba655b0>,
    'name': 'user6',
    'id': UUID('1753aeaa-2749-4ed2-8d35-6286a93fb8ab'),
    'created_at': datetime.datetime(2021, 11, 1, 14, 52, 6, 533791, 
tzinfo=datetime.timezone.utc),
    'password': 'pass'
}

<IPython.core.display.Javascript object>

In [6]:
for u in users:
    print(u.name)

user6
user7
user4
user8
user9
user10
user5
user2
user3
user1


<IPython.core.display.Javascript object>

# Query all Todos

In [7]:
with db_session() as session:
    statement = sa.select(Todo)
    results = session.execute(statement)
    todos = results.scalars().all()
todos

INFO:sqlalchemy.engine.Engine:BEGIN (implicit)
INFO:sqlalchemy.engine.Engine:SELECT todo.id, todo.created_at, todo.title, todo.content, todo.user_id, user_1.id AS id_1, user_1.created_at AS created_at_1, user_1.name, user_1.password 
FROM todo LEFT OUTER JOIN "user" AS user_1 ON user_1.id = todo.user_id
INFO:sqlalchemy.engine.Engine:[generated in 0.00130s] {}
INFO:sqlalchemy.engine.Engine:COMMIT


<IPython.core.display.Javascript object>

In [8]:
rich.print(todos[0].__dict__)

{
    '_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7feceb84ca90>,
    'title': 'Note 3',
    'created_at': datetime.datetime(2021, 11, 1, 14, 52, 6, 533791, 
tzinfo=datetime.timezone.utc),
    'user_id': UUID('c40508ee-5860-4be9-8aa1-7536572e67f6'),
    'id': UUID('25aea679-ae76-470f-b716-984a60092060'),
    'content': 'So many Notes',
    'user': <app.models.User object at 0x7feceb84cac0>
}

<IPython.core.display.Javascript object>